### Comparison of the position of an ephemeris object with astropy 

Note: jplephem module - default is DE430 while current JPL-Horizons query is based on DE440s.
casacore measures: DE200 and DE405

(from ssd.pl.nasa.gov/planets/eph_export.html)
### DE200
    Created September 1981; includes nutations but no librations.
    Referred to the dynamical equator and equinox of 2000.
    Covers JED 2305424.5 (1599 DEC 09)  to  JED 2513360.5 (2169 MAR 31).
  
### DE405
    Created May 1997; includes both nutations and librations.
    Referred to the International Celestial Reference Frame.
    Covers JED 2305424.50  (1599 DEC 09)  to  JED 2525008.50  (2201 FEB 20).
    
### DE430
    Created April 2013; includes librations and 1980 nutation.
    Referred to the International Celestial Reference Frame version 2.0.
    Covers JED 2287184.5, (1549 DEC 21) to JED 2688976.5, (2650 JAN 25).
    
### DE440
    Created June 2020; compared to DE430, about 7 years of new data have
    been added.
    Referred to the International Celestial Reference Frame version 3.0.
    Covers JED 2287184.5, (1549 DEC 31) to JED 2688976.5, (2650 JAN 25).


In [36]:
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation
from astropy.coordinates import get_body_barycentric, get_body, SkyCoord
import astropy.units as u
import shutil
from casatools import table, quanta
tb = table()
qa = quanta()

In [37]:
#!pip install ipynb
# import helper functions defined in 'Trackfield' mfs mosaic notebook
from ipynb.fs.defs.ephemimagingVenusMfsMosaicTrackfield import *

In [38]:
def ephem_dir_noazelgeo(ephemtab, refep, observatory):
    from casatools import measures
    _me = measures()
    _me.framecomet(ephemtab)
    _me.doframe(_me.observatory(observatory))
    _me.doframe(_me.epoch('utc',refep))
    return _me.measure(_me.direction('COMET'), 'ICRS')
    

In [39]:
# venus_ephem_test.ms mosaic
# obs_date - 2019/01/08/11:21:50.208000\n",
from casatasks import imhead
imgname = 'mosaic-mfs-trackfield-venus'
iminfo = imhead(imgname+'.image',mode='list')
print(iminfo['date-obs'])

2019/01/08/11:21:50.208000


In [42]:
interval = '20m'
# modify a larger time span that convers the observation to be safe
timerange='MJD 58491.0~58492.0'
if os.path.exists('Venus_JPL-Horizons_MJD58491.4.tab'):
    shutil.rmtree('Venus_JPL-Horizons_MJD58491.4.tab')
if os.path.exists('ICRS_Venus_JPL-Horizons_MJD58491.4.tab'):
    shutil.rmtree('ICRS_Venus_JPL-Horizons_MJD58491.4.tab')
getephemtable(objectname='Venus', timerange=timerange, interval=interval, 
              outfile='Venus_JPL-Horizons_MJD58491.4.tab', overwrite=True)

In [43]:
# fix posrefsys
shutil.copytree('Venus_JPL-Horizons_MJD58491.4.tab','ICRS_Venus_JPL-Horizons_MJD58491.4.tab')
tb.open('ICRS_Venus_JPL-Horizons_MJD58491.4.tab', nomodify=False)
tb.putkeyword('posrefsys','ICRS')
tb.done()
tb.open('ICRS_Venus_JPL-Horizons_MJD58491.4.tab')
print(tb.getkeyword('posrefsys'))
tb.done()

ICRS


True

In [44]:
#reformat time for astropy
t = Time('2019-01-08 11:21:50.208000')

In [45]:
loc = EarthLocation.of_site('ALMA')
print(loc)

(2225015.30883296, -5440016.41799762, -2481631.27428014) m


In [46]:
with solar_system_ephemeris.set('builtin'):
    astropy_venus = get_body('venus', t, loc)
print(astropy_venus)

<SkyCoord (GCRS: obstime=2019-01-08 11:21:50.208, obsgeoloc=(-5084912.7155837, -2955268.71328523, -2472451.221214) m, obsgeovel=(215.50599003, -370.46896024, -0.40195799) m / s): (ra, dec, distance) in (deg, deg, AU)
    (239.3651525, -16.96338202, 0.69134182)>


In [49]:
extephemtabpath='./ICRS_Venus_JPL-Horizons_MJD58491.4.tab'
intephemtabpath = './venus_ephem_test.ms/FIELD/EPHEM0_Venus_58491.4.tab'


In [53]:
# find the direction of Venus at reftm\n",
exteph_dir = ephem_dir_noazelgeo(extephemtabpath, iminfo['date-obs'], 'ALMA')
inteph_dir = ephem_dir_noazelgeo(intephemtabpath, iminfo['date-obs'], 'ALMA')

In [54]:
print('internal ephem:',qa.time(inteph_dir['m0'],prec=9)[0],qa.angle(inteph_dir['m1'],prec=9)[0])
print('external ephem:',qa.time(exteph_dir['m0'],prec=9)[0],qa.angle(exteph_dir['m1'],prec=9)[0])

print('astropy(builtin):',qa.time(qa.quantity(astropy_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(astropy_venus.dec.deg, 'deg'), prec=9)[0])

internal ephem: 15:57:28.347 -016.57.51.787
external ephem: 15:57:27.953 -016.57.53.509
astropy(builtin): 15:57:27.637 -016.57.48.175


In [55]:
solar_system_ephemeris.set('jpl') 


<ScienceState solar_system_ephemeris: 'jpl'>

In [56]:
de440_venus = get_body('venus', t, loc, ephemeris='de440')

In [58]:
print('astropy(de440):',qa.time(qa.quantity(de440_venus.ra.deg, 'deg'), prec=9)[0], qa.angle(qa.quantity(de440_venus.dec.deg, 'deg'), prec=9)[0])

astropy(de440): 15:57:27.564 -016.57.47.299


In [59]:
pwd

'/Users/ttsutsum/SWDevel/VSCodeCasa/ephemimscripts'